In [16]:
from sklearn import svm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from scipy.stats import zscore
from sklearn.metrics import f1_score,roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay,precision_score,recall_score

from sklearn.model_selection import GridSearchCV

In [7]:
#same scaler method
import preprocessing

# preprocessing.split_external_data(df)
# undersample_data(df)
# rescale_dataset(df)
# remove_outliers(df)
# preprocess(df, save_csv = False)
df=pd.read_csv("C:\\Users\\weihua\\Desktop\\Python_MSIS_SYS1\\finalproject\\sbcdata.csv")

df_uml, df_umg = preprocessing.preprocess(df, save_csv = False)

# print('training set preprocessing confirmation')
# display(df_uml.Diagnosis.value_counts()) #make sure data is balance
# display(df_uml)

# print('validation set preprocessing confirmation')
# display(df_umg.Diagnosis.value_counts()) #make sure data is balance
# display(df_umg)

y=df_umg['Diagnosis']
x=df_umg[['Age','Sex','HGB','MCV','PLT','RBC','WBC',]]
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=42)

# 创建SVM模型实例 model = svm.SVC(kernel='rbf', C=1.0, gamma='scale')
# model = svm.SVC(kernel='rbf', C=0.1, gamma='scale', class_weight='balanced')
# 训练模型
# model.fit(X_train, y_train)
# 在测试集上预测
# y_pred = model.predict(X_test)
# 评估模型
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

# param_grid = {'C': [0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1]}
# grid = GridSearchCV(svm.SVC(), param_grid, cv=3)
# grid.fit(X_train, y_train)

# #optional 2
# y_pred2=grid.predict(X_test)
# f1 = f1_score(y_test, y_pred2)
# #f1 score for optiona 2
# f1 = f1_score(y_test, y_pred2)
# f1_macro = f1_score(y_test, y_pred2, average='macro')
# print("F1 Score (Macro Average):", f1_macro)

#f1 score
# f1 = f1_score(y_test, y_pred)
# f1_macro = f1_score(y_test, y_pred, average='macro')
# print("F1 Score (Macro Average):", f1_macro)


Count of each target variable before preprop: Diagnosis
0.0    1776380
1.0     251395
Name: count, dtype: int64
Count of each target variable after dropping na and duplicates: Diagnosis
0.0    1673822
1.0     243123
Name: count, dtype: int64
Length of X_df after scaling: 1916945
Count of each target variable after rescaling: Diagnosis
0.0    1673822
1.0     243123
Name: count, dtype: int64
Count of each target variable before undersampling: Diagnosis
0.0    1673822
1.0     243123
Name: count, dtype: int64
Count of each target variable after undersampling: Diagnosis
0.0    243123
1.0    243123
Name: count, dtype: int64
Count of each target variable before preprop: Diagnosis
0.0    526943
1.0    128578
Name: count, dtype: int64
Count of each target variable after dropping na and duplicates: Diagnosis
0.0    513133
1.0    122511
Name: count, dtype: int64
Length of X_df after scaling: 635644
Count of each target variable after rescaling: Diagnosis
0.0    513133
1.0    122511
Name: count, d

In [18]:
#MZ original code, 处理整个数据集时间过长
# df1=pd.read_csv("C:\\Users\\weihua\\Desktop\\Python_MSIS_SYS1\\finalproject\\sbcdata.csv")
# print(df1.head())

df=pd.read_csv("C:\\Users\\weihua\\Desktop\\Python_MSIS_SYS1\\finalproject\\sbcdata_5000.csv")
df.drop(columns=['Id', 'Center','Set','Sender','Episode','Time','TargetIcu','SecToIcu','PCT'],inplace=True)

# 打印头测试 print(df.head())
print(df.dtypes)
print(df.shape)
print(df.isna().sum())
df['CRP']=df['CRP'].fillna(df['CRP'].mean())
#删除missing数据 drop
df.dropna(inplace=True)
print(df.isna().sum())
#重新处理sex和diagnosis列，因为不是数字 encode, handle sex and diagnosis
label_encoder_sex = LabelEncoder()
df['Sex'] = label_encoder_sex.fit_transform(df['Sex'])  # Male -> 1, Female -> 0
label_encoder_diagnosis = LabelEncoder()
df['Diagnosis'] = label_encoder_diagnosis.fit_transform(df['Diagnosis'])  # Positive -> 1, Negative -> 0

#数据分层，提取使用的columns SVM
y=df['Diagnosis']
x=df[['Age','Sex','CRP','HGB','MCV','PLT','RBC','WBC',]]
# 打印头测试 print(x.head(),y.head())
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=42)

# 标准化数据 Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 创建SVM模型，使用RBF核
# 创建SVM模型实例 model = svm.SVC(kernel='rbf', C=1.0, gamma='scale')
model = svm.SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced')
# 训练模型
model.fit(X_train, y_train)
# 在测试集上预测
y_pred = model.predict(X_test)
# 评估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
precision = precision_score(y_test, y_pred,average='weighted')
print(f"Precision: {precision:.2f}")
recall = recall_score(y_test, y_pred,average='weighted')
print(f"Recall: {recall:.2f}")
# print("Classification Report:\n", classification_report(y_test, y_pred))
#f1 score 
# f1 = f1_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='weighted')
print("F1 Score (Macro Average):", f1_macro)

# 计算 ROC 曲线
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
# 计算 AUC
roc_auc = auc(fpr, tpr)
print(f"AUC: {roc_auc:.2f}")
# 绘制 ROC 曲线
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # 随机分类器的线
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()



Age            int64
Sex           object
Diagnosis     object
CRP          float64
HGB          float64
MCV          float64
PLT          float64
RBC          float64
WBC          float64
dtype: object
(5001, 9)
Age             0
Sex             0
Diagnosis       0
CRP          2570
HGB           180
MCV           180
PLT           180
RBC           180
WBC           180
dtype: int64
Age          0
Sex          0
Diagnosis    0
CRP          0
HGB          0
MCV          0
PLT          0
RBC          0
WBC          0
dtype: int64
Accuracy: 0.7461139896373057
Precision: 0.86
Recall: 0.75
F1 Score (Macro Average): 0.7870416630755885


ValueError: multiclass format is not supported